## Create models for real time inference



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
%mkdir infer_mt
%cd infer_mt

In [ ]:
!pip install -q gdown
import gdown
# must modify id for large file
id = "10i7HAA9UvWPDmi4iTT04E1rb1y4BmXIX&export=download&confirm=t"
output="model.bin"
gdown.download(id=id, output=output, quiet=False)

In [ ]:
!pip install -q ctranslate2 OpenNMT-py==2.2.0 sentencepiece

In [ ]:
# Try the following
!pip install -q ctranslate2 OpenNMT-py sentencepiece
!pip install -q gradio transformers torch

In [ ]:
import gradio as gr
import ctranslate2
import sentencepiece as spm

In [ ]:
build_directory=!pwd
input_model="model.bin"
mtmodel="enga"

In [ ]:
!ct2-opennmt-py-converter --model_path $input_model --output_dir $mtmodel

## Launch Gradio and run translation interface

In [ ]:
LANGS = ["eng_Latn", "gle_Latn"]
MODEL_LIST = ["enga"]

def translate(text, src_lang, tgt_lang, cmodel):
    """ Translate the text from source lang to target lang """
    translator = ctranslate2.Translator(str(cmodel),device="cpu")

    sp = spm.SentencePieceProcessor(str(cmodel)+"/spm.model")
    input_tokens = sp.encode(text, out_type=str)
    results = translator.translate_batch([input_tokens])
    output_tokens = results[0].hypotheses[0]
    result = sp.decode(output_tokens)

    return result

In [ ]:
demo = gr.Interface(
    fn=translate,

     inputs=[
        gr.components.Textbox(label="Text"),
        gr.components.Dropdown(label="Source Language", choices=LANGS),
        gr.components.Dropdown(label="Target Language", choices=LANGS),
        gr.components.Dropdown(label="Choose Model", choices=MODEL_LIST),
    ],

    outputs=["text"],
    examples=[["I would love to go to the beach today", "eng_Latn", "gle_Latn"],
              ["Tomorrow, I will have lunch in the park", "eng_Latn", "gle_Latn"]],
    cache_examples=False,
    title="Translation Automation Demo",
    description="To record any interesting outputs in flagged/log.csv, just hit the Flag button"
)

In [ ]:
demo.launch(debug=True)